In [299]:
import pathlib
import random
import pandas as pd
import numpy as np
import sys
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict

from sklearn.metrics import (
    f1_score, 
    accuracy_score,
    classification_report, 
)

ROOT_DIR = pathlib.Path().absolute()
DATA_DIR = ROOT_DIR / "data"
RANDOM_SEED = 42

## Загрузка и обзор данных

In [300]:
df_trends = pd.read_csv(DATA_DIR / "trends_description.csv")
df = pd.read_csv(DATA_DIR / "train.csv")
df_test = pd.read_csv(DATA_DIR / "test.csv")

In [301]:
df.head()

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Обучение моделей

### Предобработка данных

In [302]:
df.head()

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [303]:
X, y = df[["text"]], df[[f"trend_id_res{i}" for i in range(50)]]
X = X.astype("str").copy()
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size= 0.8, random_state = 42)
print(f"X_train.shape is {X_train.shape}")
print(f"y_train.shape is {y_train.shape}")
print(f"X_val.shape is {X_val.shape}")
print(f"y_val.shape is {y_val.shape}")
X_test = df_test[["text"]]
print(f"X_test.shape is {X_test.shape}")
trend_info = pd.read_csv(DATA_DIR / "trends_description.csv")


X_train.shape is (3698, 1)
y_train.shape is (3698, 50)
X_val.shape is (925, 1)
y_val.shape is (925, 50)
X_test.shape is (9015, 1)


In [304]:
X.shape


(4623, 1)

In [305]:
X_train

,text
1538,"Ну, за [NUM]ч. и [NUM] мин. мне ещё никогда не..."
2991,Доставка всегда осуществляется значительно дол...
2812,Задержка доставки
4515,"Отличный сервис, только бы ассортимент расшири..."
4531,"Поддержка говно, курьеры опаздывают минут на [..."
...,...
4426,"+ быстро. - иногда сумма заказа очень велика, ..."
466,🦉
3092,До самоката я тратил меньше денег в день
3772,О вас редко думаю. Напрягает СТМ. Когда непоня...


In [306]:

categories = []
for i in trend_info['trend']:
    categories.append(i)
categories

['Долгая доставка',
 'Доставка стала долгой',
 'Время доставки не соответствует заявленому',
 'Регулярные опоздания',
 'Не отследить реальное время доставки',
 'Курьер на карте',
 'Нет доставки по адресу',
 'Не предупреждаем об удалении товара',
 'Высокая минимальная сумма заказа',
 'Сумма заказа меняется во время набора корзины',
 'Минимальная сумма заказа',
 'Товары с подходящим сроком годности',
 'Высокие цены',
 'Не довезли товар',
 'Товар испорчен во время доставки',
 'Просроченные товары',
 'Замечания по работе курьеров',
 'Не читаем комментарии',
 'Спасибо',
 'Нет смысла',
 'Всё нормально',
 'Всё плохо',
 'Скидки для постоянных клиентов',
 'Больше акций/скидок',
 'Скидка/промокод распространяется не на все товары',
 'Непонятно как работает скидка',
 'Не сработала скидка/акция/промокод',
 'Качество товаров',
 'Маленький ассортимент',
 'Нет в наличии товара',
 'Качество поддержки',
 'Замечания по работе сборщика',
 'Отменили заказ',
 'Знание русского языка',
 'Привезли чужой заказ

In [307]:
import re
import string

def lower_text(text: str):
    return text.lower()

def remove_numbers(text: str):
    """
    Substitute all punctuations with space in case of
    "there is5dogs".

    If subs with '' -> "there isdogs"
    With ' ' -> there is dogs
    """
    text_nonum = re.sub(r'\d+', ' ', text)
    return text_nonum

def remove_punctuation(text: str):
    """
    Substitute all punctuations with space.

    Example:
    "hello!nice to meet you" -> "hello nice to meet you"
    """
    # Create a translation table that maps each punctuation to a space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

    # Use translate to substitute punctuation with spaces
    text_nopunct = text.translate(translator)

    return text_nopunct



def remove_multiple_spaces(text: str):
    """
    Replaces multiple spaces with a single space.

    Example:
    "This  is   a   test" -> "This is a test"
    """
    # Use regex to substitute multiple spaces with a single space
    text_no_doublespace = re.sub(r'\s+', ' ', text).strip()

    return text_no_doublespace

In [308]:
sample_text = X_train['text'][22]

_lowered = lower_text(sample_text)
_without_numbers = remove_numbers(_lowered)
_without_punct = remove_punctuation(_without_numbers)
_single_spaced = remove_multiple_spaces(_without_punct)

print(sample_text)
print('-'*10)
print(_lowered)
print('-'*10)
print(_without_numbers)
print('-'*10)
print(_without_punct)
print('-'*10)
print(_single_spaced)

Заказ на сумму от [NUM]₽ это дорого, было бы лучше как было раньше [NUM]₽
----------
заказ на сумму от [num]₽ это дорого, было бы лучше как было раньше [num]₽
----------
заказ на сумму от [num]₽ это дорого, было бы лучше как было раньше [num]₽
----------
заказ на сумму от  num ₽ это дорого  было бы лучше как было раньше  num ₽
----------
заказ на сумму от num ₽ это дорого было бы лучше как было раньше num ₽


In [309]:
# Importing necessary modules from NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
# Tokenization function
def tokenize_text(text: str) -> list[str]:
    """
    Tokenizes input text into a list of words.
    """
    return nltk.tokenize.word_tokenize(text)

# Remove stopwords function
def remove_stop_words(tokenized_text: list[str]) -> list[str]:
    """
    Removes stopwords from a list of tokenized words.
    """
    # Ensure stopwords are downloaded
    nltk.download('stopwords')
    stop_words = set(stopwords.words('russian'))

    return [word for word in tokenized_text if word.lower() not in stop_words]

# Stemming function
def stem_words(tokenized_text: list[str]) -> list[str]:
    """
    Stems words in the tokenized text using PorterStemmer.
    """
    stemmer = SnowballStemmer("russian") 
    return [stemmer.stem(word) for word in tokenized_text]




[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [310]:
_tokenized = tokenize_text(_single_spaced)
_without_sw = remove_stop_words(_tokenized)
_stemmed = stem_words(_without_sw)

print(_single_spaced)
print('-'*10)
print(_tokenized)
print('-'*10)
print(_without_sw)
print('-'*10)
print(_stemmed)

заказ на сумму от num ₽ это дорого было бы лучше как было раньше num ₽
----------
['заказ', 'на', 'сумму', 'от', 'num', '₽', 'это', 'дорого', 'было', 'бы', 'лучше', 'как', 'было', 'раньше', 'num', '₽']
----------
['заказ', 'сумму', 'num', '₽', 'это', 'дорого', 'раньше', 'num', '₽']
----------
['заказ', 'сумм', 'num', '₽', 'эт', 'дор', 'раньш', 'num', '₽']


[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [311]:
def preprocessing_stage(text):
    _lowered = lower_text(text)
    _without_numbers = remove_numbers(_lowered)
    _without_punct = remove_punctuation(_without_numbers)
    _single_spaced = remove_multiple_spaces(_without_punct)
    _tokenized = tokenize_text(_single_spaced)
    _without_sw = remove_stop_words(_tokenized)
    _stemmed = stem_words(_without_sw)

    return _stemmed

def clean_text_inplace(df):
    df['text'] = df['text'].apply(preprocessing_stage)
    return df
def preprocess(df):

    df.fillna(" ", inplace=True)
    _cleaned = clean_text_inplace(df)
    
    #for i in range(len(df)):
    #    if i==8:
    #        continue
        #print(df['text'][i])
    #    df['text'][i]  = preprocessing_stage(df['text'][i])
    #    print(df['text'][i])
        
    return _cleaned




In [312]:
X_train['text']

1538    Ну, за [NUM]ч. и [NUM] мин. мне ещё никогда не...
2991    Доставка всегда осуществляется значительно дол...
2812                                    Задержка доставки
4515    Отличный сервис, только бы ассортимент расшири...
4531    Поддержка говно, курьеры опаздывают минут на [...
                              ...                        
4426    + быстро. - иногда сумма заказа очень велика, ...
466                                                     🦉
3092             До самоката я тратил меньше денег в день
3772    О вас редко думаю. Напрягает СТМ. Когда непоня...
860     Доставка стала очень долгая, цены завышены чем...
Name: text, Length: 3698, dtype: object

In [313]:
train_preprocessed = preprocess(X)
test_preprocessed = preprocess(X_test)

train_preprocessed.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\Katie\miniconda3\envs\mltest\nltk_data...
[nltk_data]   Package stopwords is already up-to-date

,text
0,"[маленьк, выбор, товар, хотел, ассортимент, врод]"
1,[быстр]
2,"[доставк, постоя, задержива]"
3,"[наценк, ассортимент, расстраива]"
4,"[немн, скинут, минимальн, сумм, заказ, оплачив..."


In [315]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size= 0.8, random_state = 42)

In [318]:
y_val.shape

(925, 50)

In [331]:
!pip install torchtext

In [325]:
for _, sample in X_train.iterrows():
    print(sample.to_list()[0])
    break

['num', 'ч', 'num', 'мин', 'ещ', 'доставля', 'заказ', 'отвратительн', 'одн', 'слов']


In [332]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(df):
    for _, sample in X_train.iterrows():
        yield sample.to_list()[0]


# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

vocab = ''
vocab = build_vocab_from_iterator(yield_tokens(X), specials=special_symbols)
vocab.set_default_index(UNK_IDX)

OSError: [WinError 127] The specified procedure could not be found

In [319]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn

class Neuraln(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        # Direct LSTM initialization without embedding layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, num_layers=1)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, text, h=None, c=None):
        out = 0
        if h is not None and c is not None:
            out, (h, c) = self.lstm(text, (h, c))
        else:
            out, (h, c) = self.lstm(text)

        x = self.fc(out)
        return x[1]  # Assuming the use of the second time step output as before


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, padding_idx=1):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim,
            padding_idx=padding_idx        )
        self.fc = nn.Linear(embed_dim, num_classes)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        """
        Args:
            text: Tensor of token indices concatenated for the batch
            offsets: Tensor indicating the start index of each sequence in `text`
        Returns:
            Logits for each class
        """
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [ ]:
# Parameters
vocab_size = len(vocab)
embed_dim = 128
num_classes = 50

# Instantiate the model
#model = LSTMTextClassificationModel(vocab_size, embed_dim, hidden_dim = 128, num_classes = num_classes)
model = TextClassificationModel(vocab_size, embed_dim, num_classes)
#model = Neuraln(vocab_size, embed_dim=128, hidden_dim = 8, num_classes = num_classes)
# Example input
text = torch.tensor([vocab[token] for token in ["a", "simple"]], dtype=torch.long)
offsets = torch.tensor([0], dtype=torch.long)  # Single sequence starting at index 0

# Forward pass
logits = model(text, offsets)
print(logits)


In [298]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), lowercase=False,max_features=1000)
vectors=vectorizer.fit_transform(X_train['text'])
vectors.shape

TypeError: expected string or bytes-like object, got 'list'

In [10]:
vectorizer.get_feature_names_out()


array(['NUM', 'NUM NUM', 'NUM NUM NUM', 'NUM NUM мин', 'NUM NUM минут',
       'NUM NUM часа', 'NUM дня', 'NUM до', 'NUM мин', 'NUM минут',
       'NUM минут NUM', 'NUM минут доставка', 'NUM минут заказ',
       'NUM минут не', 'NUM минут но', 'NUM минут по', 'NUM минут сейчас',
       'NUM минут то', 'NUM минут это', 'NUM минуты', 'NUM раз',
       'NUM раза', 'NUM руб', 'NUM рублей', 'NUM час', 'NUM часа',
       'NUM часов', 'NUM это', 'Ассортимент', 'Бывает', 'Было', 'Было бы',
       'Быстрая', 'Быстрая доставка', 'Быстро', 'Вместо', 'Время',
       'Время доставки', 'Все', 'Все отлично', 'Все супер', 'Все хорошо',
       'Все хорошо но', 'Всегда', 'Всё', 'Вы', 'Да', 'Долгая',
       'Долгая доставка', 'Долго', 'Долго доставляют', 'Дорого',
       'Доставка', 'Доставка быстрая', 'Доставка очень', 'Доставка стала',
       'Думаю', 'Думаю что', 'Если', 'Если бы', 'Жду', 'За', 'Заказ',
       'Заказы', 'Зачем', 'Из', 'Иногда', 'Каждый', 'Каждый раз', 'Как',
       'Качество', 'Качест

In [11]:
dense_vectors = vectors.todense()
dense_vectors.shape

(3698, 1000)

In [12]:
from sklearn.svm import SVC
preprocessor = ColumnTransformer(
    [
        ("vetorizer", TfidfVectorizer(analyzer="char_wb", ngram_range = (1,3), lowercase=False,max_features=1000), "text")
    ],                         
    remainder = "passthrough"
)

pipeline_multiout = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clf", MultiOutputClassifier(SVC(kernel='rbf', gamma=0.1, C=2))),
    ]
)
display(pipeline_multiout)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vetorizer',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  lowercase=False,
                                                                  max_features=1000,
                                                                  ngram_range=(1,
                                                                               3)),
                                                  'text')])),
                ('clf', MultiOutputClassifier(estimator=SVC(C=2, gamma=0.1)))])

In [13]:
cross_valid = cross_validate(pipeline_multiout, 
                             X_train, y_train, 
                             cv = 5, scoring = ["accuracy"], n_jobs = -1)
print("test_accuracy:", cross_valid["test_accuracy"].mean())

test_accuracy: 0.23363639688402885


In [14]:
y_pred = cross_val_predict(pipeline_multiout, X_train, y_train, cv = 2)

In [15]:
# Посмотрим на различные метрики
print(classification_report(y_train, y_pred, zero_division = 0))

              precision    recall  f1-score   support

           0       0.88      0.41      0.56       661
           1       0.50      0.01      0.01       270
           2       0.76      0.31      0.45       486
           3       0.90      0.19      0.31       289
           4       0.00      0.00      0.00       108
           5       0.00      0.00      0.00        44
           6       0.00      0.00      0.00        16
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00       109
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        76
          11       0.00      0.00      0.00        87
          12       0.95      0.51      0.67       491
          13       0.00      0.00      0.00        29
          14       0.00      0.00      0.00        62
          15       0.00      0.00      0.00        66
          16       0.00      0.00      0.00       166
          17       0.00    

In [16]:
# Посмотрим на целевую метрику
accuracy_score(y_train, y_pred)

0.19226608977825851

###  Тренировка окончательной модели

In [17]:
pipeline_multiout.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vetorizer',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  lowercase=False,
                                                                  max_features=1000,
                                                                  ngram_range=(1,
                                                                               3)),
                                                  'text')])),
                ('clf', MultiOutputClassifier(estimator=SVC(C=2, gamma=0.1)))])

##  Предсказание и загрузка решения

In [18]:
pred_test = pipeline_multiout.predict(df_test[["text"]].astype("str"))

In [19]:
res = pd.DataFrame(np.hstack([df_test["index"].values.reshape(df_test.shape[0], 1), pred_test]),
                  columns = ["index"]+[f"{i}" for i in range(50)])

In [26]:
type(res['index'][1])

numpy.int64

In [20]:
res.head()

,index,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,3135,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22118,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23511,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:

res["target"] = res.drop('index',axis=1).apply(lambda r: r.index[r.ne(0)].to_list(), axis=1)


res

,index,0,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,target
0,3135,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
1,4655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[12]
2,22118,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[2]
3,23511,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[0]
4,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9010,3523,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
9011,24925,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
9012,6327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
9013,530,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]


In [22]:
f=open('submission.csv', 'w')
f.write("index,target")
f.write("\n")
for i in range(len(res)):
    
    f.write(str(res['index'][i]))
    f.write(',')
    for x in res['target'][i]:
        f.write(str(x))
        f.write(' ')
    f.write("\n")

f.close()

In [23]:
res.iloc[:, 1:].sum()

0                                                       706
1                                                        87
2                                                       718
3                                                       243
4                                                         0
5                                                         0
6                                                         0
7                                                         0
8                                                         0
9                                                         0
10                                                        0
11                                                       29
12                                                      831
13                                                        0
14                                                        0
15                                                        0
16                                      

In [24]:
res["0"].value_counts()

0
0    8309
1     706
Name: count, dtype: int64

In [25]:
type(res['index'][1])

numpy.int64